In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-06-16


Found rundate 2024-06-15 10:00:03 < 2024-06-16 at commit ed45e65e; returning commit 267f241


'2024-06-16 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-06-16 10:00:03, NJSP was reporting 293 YTD deaths
Current YTD Deaths (2025-06-16 10:00:02-04:00): 236
Previous year YTD Deaths (adjusted; 2024-06-16 10:00:03): 292.0473635989467
Projected 2025 total: 592.7530141407564
-19.2% change


3ae31fbf71ab25641aef4d15e144755f81e0ee3c: FAUQStats cache miss: 2024, Sun Jun 16 10:00:03 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,NaN,State Highway 27,1.0,0.0,0.0,1.0,0.0,27,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13257,12,Middlesex,1213,Monroe Twsp,NaN,County 522 MP 11.3,1.0,0.0,0.0,1.0,0.0,522,NaN,2024-06-12 00:05:00-04:00
13259,12,Middlesex,1213,Monroe Twsp,NaN,County 619 MP 2.01,1.0,1.0,0.0,0.0,0.0,619,NaN,2024-06-12 10:06:00-04:00
13260,01,Atlantic,0113,Hammonton Town,8th street,8th street,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-06-13 16:56:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,9,1,5,0,14
Bergen,8,1,2,0,11
Burlington,11,4,3,0,17
Camden,10,2,6,0,17
Cape May,3,0,0,0,3
Cumberland,5,2,2,0,8
Essex,6,1,9,1,17
Gloucester,7,4,2,0,10
Hudson,2,1,3,1,6


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

45.6% through the year, 54.4% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,13,31,9
Bergen,driver,9,14,8
Burlington,driver,16,31,11
Camden,driver,10,19,10
Cape May,driver,1,8,3
...,...,...,...,...
Salem,crashes,6,14,7
Somerset,crashes,5,18,6
Sussex,crashes,3,12,1


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,38,1,24,2,14
Bergen,27,1,13,2,10
Burlington,39,1,24,5,12
Camden,29,0,19,3,9
Cape May,18,0,16,0,1
Cumberland,25,0,16,6,9
Essex,43,2,16,6,25
Gloucester,31,0,18,13,6
Hudson,23,2,10,1,14


In [9]:
projected.to_csv(PROJECTED_CSV)